In [1]:
import numpy as np
#import pandas as pd

import tensorflow.keras as keras
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image
from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

from ultralytics import YOLO
from matplotlib.pyplot import imshow
import pandas as pd

In [2]:
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Webcam Images',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 42,
   
    color_mode = 'grayscale'
    )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Webcam Images',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 42,
   # class_names = ['Happy', 'Sad', 'Neutral'],
    color_mode = 'grayscale'
    )

Found 164 files belonging to 3 classes.
Using 132 files for training.


Found 164 files belonging to 3 classes.
Using 32 files for validation.


In [3]:
#convert to numpy images and then perform PCA to extract the values
#convert images to numpy
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [4]:
#flatten the images
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1] * X_val.shape[2]))

In [5]:
#Map the high dimensional images to a lower dimension
pca = PCA(n_components = 50)

X_train_lower_dim = pca.fit_transform(X_train)

In [6]:
pca.explained_variance_ratio_.sum()

#As can be seen from the output below, the first 50 components explain a substantial proportsion of the variance
#include train accuracy as well

0.9429681

In [7]:
#fitting an SVC object on this

rf = RandomForestClassifier()

rf.fit(X_train_lower_dim, np.argmax(y_train, axis = 1))

RandomForestClassifier()

In [8]:
#map the validation set to the lower dimension and then predict using the fitted rf model

X_val_lower_dim = pca.transform(X_val)
y_pred = rf.predict(X_val_lower_dim)

In [9]:
#computing the accuracy score 

#gives an accuracy score of 0.7! Significantly higher than other classifiers like SVM
accuracy_score(np.argmax(y_val, axis = 1),y_pred)

0.78125

### With face cropping

In [11]:
#try the above but with cropping the face
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Webcam Images',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 42,
   )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Webcam Images',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 42,
  
    )

Found 164 files belonging to 3 classes.
Using 132 files for training.
Found 164 files belonging to 3 classes.
Using 32 files for validation.


In [12]:
#convert to numpy images and then perform PCA to extract the values
#convert images to numpy
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [13]:
#convert the images to cropped faces
face_classifier = YOLO("../../app/src/face_detection/model/yolov8n-face.pt")

X_cropped_train = []
y_train_cropped = []

n_train = X_train.shape[0]
for i in range(n_train):
    face = X_train[i]
   
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((200, 200))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_cropped_train.append(numpy_cropped_face)
        y_train_cropped.append(y_train[i])

X_val_cropped = []
y_val_cropped = []
n_val = X_val.shape[0]

for i in range(n_val):
    face = X_val[i]
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((200, 200))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_val_cropped.append(numpy_cropped_face)
        y_val_cropped.append(y_val[i])


c:\Users\veena\OneDrive\Desktop\Nus\Y4S2\IS4242\Project Actual\moodify\emotion_detection\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 640x640 1 face, 275.1ms
Speed: 164.1ms preprocess, 275.1ms inference, 2098.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 134.2ms
Speed: 151.4ms preprocess, 134.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 140.3ms
Speed: 130.2ms preprocess, 140.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 120.3ms
Speed: 175.4ms preprocess, 120.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 125.2ms
Speed: 94.4ms preprocess, 125.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 147.8ms
Speed: 17.4ms preprocess, 147.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 135.7ms
Speed: 27.1ms preprocess, 135.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 141.2ms
Speed: 19.6ms preprocess, 141.2ms inference, 0.0ms postprocess per image at s

In [14]:
#convert the lists to np arrays
X_cropped_train = np.array(X_cropped_train)

X_val_cropped = np.array(X_val_cropped)



In [15]:

#Flatten the RGB image into a flat tensor


X_cropped_train = X_cropped_train.reshape((X_cropped_train.shape[0], X_cropped_train.shape[1] * X_cropped_train.shape[2] * X_cropped_train.shape[3]))
X_val_cropped = X_val_cropped.reshape((X_val_cropped.shape[0], X_val_cropped.shape[1] * X_val_cropped.shape[2] * X_val_cropped.shape[3]))


y_train_lab = np.array(y_train_cropped).argmax(axis = 1)
y_val_lab = np.array(y_val_cropped).argmax(axis = 1)


In [16]:
#apply PCA to map to a lower dimension
pca = PCA(n_components = 50)

X_train_lower_dim = pca.fit_transform(X_cropped_train)

In [17]:
rf_cropped = RandomForestClassifier()

#train on the lower dimensions of the data
rf_cropped.fit(X_train_lower_dim, y_train_lab)

RandomForestClassifier()

In [18]:
#compute training accuracy
y_train_pred = rf_cropped.predict(X_train_lower_dim)

#acheives perfect score on the training data, could be a sign of overfitting
print(accuracy_score(y_train_lab, y_train_pred))

1.0


In [19]:
#map the validation data to a lower dimension based on the data
X_val_cropped_lower_dim = pca.transform(X_val_cropped)

In [20]:
y_pred = rf_cropped.predict(X_val_cropped_lower_dim)
#display the accuracy of the model
print(accuracy_score(y_val_lab, y_pred))
#note that the accuracy is lower compared to not using the cropped image

0.5862068965517241
